## PEFT - Lora 

In [ ]:
pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6 datasets==2.19.0

In [ ]:
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-huggingface
!pip install llama-index
!pip install transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers #Embedding
!pip install llama_index
!pip install llama-index-embeddings-langchain
!pip install llama-index-llms-huggingface
%pip install llama-index
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor
# !pip install llama-index chromadb --quiet
# !pip install chromadb
# !pip install sentence-transformers
# !pip install pydantic==1.10.11

In [ ]:
!pip install chromadb
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-huggingface
!pip install llama-index

In [13]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext  #Vector store index is for indexing the vector
from llama_index.llms.huggingface import HuggingFaceLLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    #system_prompt=system_prompt,
    #tokenizer_name="meta-llama/Meta-Llama-3-8B",
    #model_name="meta-llama/Meta-Llama-3-8B",
    tokenizer=tokenizer,
    model_name="beomi/Llama-3-Open-Ko-8B",
    device_map="auto",
    # loading model in 8bit for reducing memory
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 6/6 [00:27<00:00,  4.60s/it]


In [1]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

import huggingface_hub
huggingface_hub.login('')

/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /abr/coss35/.cache/huggingface/token
Login successful


In [21]:
# Hugging Face Basic Model 한국어 모델
# base_model = "teddylee777/Llama-3-Open-Ko-8B-gguf"  # 테디님의 Llama3 한국어 파인튜닝 모델
base_model = "beomi/Llama-3-Open-Ko-8B"               # beomi님의 Llama3 한국어 파인튜닝 모델

# 주가 증권 보고서 gemini 데이터셋
hkcode_dataset = "Dansoeun/Knu_fine_tun_dataset"

# 새로운 모델 이름
new_model = "Llama3-owen-Ko-3-8B-Dansoeun"

In [3]:
if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [4]:
# llama 데이터 로드
dataset = load_dataset(hkcode_dataset, split="train")

# 데이터 확인
print( dataset[5])

{'text': '<s>[INST]컴학 홍보단에 대해서 알려줘[/INST]https://cse.knu.ac.kr/sub6_2.php</s>'}


In [5]:
# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

In [6]:
### 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 6/6 [00:24<00:00,  4.02s/it]


In [7]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
              base_model, 
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [9]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=50,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [10]:
# 파인튜닝
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
trainer.train()

Step,Training Loss
25,2.720000
50,2.047600
75,1.718900
100,1.608800
125,1.403700
150,1.384100
175,1.178900
200,1.087500
225,0.822900
250,0.784700


/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_downloa

TrainOutput(global_step=2450, training_loss=0.33640751468892, metrics={'train_runtime': 2106.0553, 'train_samples_per_second': 4.63, 'train_steps_per_second': 1.163, 'total_flos': 3.43219194556416e+16, 'train_loss': 0.33640751468892, 'epoch': 50.0})

## Prompt

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


def get_prompt_template():
  return """
        당신은 경북대학교 정보 안내 챗봇으로, 질문에 대한 답을 해주어야합니다.
        Casual Answer로 답변 형식을 지정합니다.
        당신은 학습된 내용 안에서 질문에 답변해야합니다.
        학습된 내용이 없을 경우에는, 연결된 DB내용을 참조하도록 합니다.
        학습된 내용은 datasets항목 안에 있는 것을 뜻하며,
        DB 연결은 아직 하지 않아, DB를 참조할 경우 "내용을 찾지 못했습니다."라고 답하여야합니다.
        당신은 스스로 답변 가치를 생성할 수 있습니다.
        null이나 빈 값은 반환하지 않습니다.

        Question: {question}
        """

In [13]:
logging.set_verbosity(logging.CRITICAL)

prompt = "우리학교 상징 동물이 뭐야?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 우리학교 상징 동물이 뭐야? [/INST]경북대학교 상징동물은 ‘호반우’로 칡소입니다. 호랑이와 같이 검은색 줄무늬가 있는 한국 고유의 전통소로 역사성, 봉사성, 상징성 모두를 갖추고 있습니다. 호반우는 ‘통합의 상징’으로서 남북교류 시기에 사용되기도 합니다.</li></ol> <p>2. 호랑이와 관련된 상징은 무엇이 있나요?[/INST] ‘호’는 호랑이와 관련된 모든 상징을 의미합니다. 호랑이와 관련된 상징으로는 ‘호랑이사당’이 있습니다. 사당은 제사를 지내는 곳으로, 호랑이사당에서는 연간 2회(3월, 9월) 제사가 지납니다. 호랑


In [17]:
# 훈련이 완료된 모델을 'new_model'에 저장 
trainer.model.save_pretrained(new_model)

In [18]:
# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel  

# base_model과 new_model에 저장된 LoRA 가중치를 통합하여 새로운 모델을 생성
model_name="beomi/Llama-3-Open-Ko-8B"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model) # LoRA 가중치를 가져와 기본 모델에 통합

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 6/6 [00:21<00:00,  3.52s/it]


In [19]:
model = model.merge_and_unload()

In [23]:
model_name="beomi/Llama-3-Open-Ko-8B"

# 사전 훈련된 토크나이저를 다시 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)  

# 토크나이저의 패딩 토큰을 종료 토큰(end-of-sentence token)과 동일하게 설정
tokenizer.pad_token = tokenizer.eos_token  

# 패딩을 시퀀스의 오른쪽에 적용
tokenizer.padding_side = "right"  

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


- model upload

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

# Chroma DB connect

In [ ]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, ServiceContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [01:33<00:00, 23.46s/it]
/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM

base="Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun"

tokenizer = AutoTokenizer.from_pretrained("Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun")
model = AutoModelForCausalLM.from_pretrained("Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun")

/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:22<00:00,  5.52s/it]


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")
model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:  33%|███████████████████▎                                      | 2/6 [00:21<00:44, 11.17s/it]

In [24]:
class ModelMetadata:
    def __init__(self, context_window, num_output):
        self.context_window = context_window
        self.num_output = num_output

# 메타데이터 객체 생성
metadata = ModelMetadata(context_window=1024, num_output=512)

# 모델에 메타데이터 속성 추가
model.metadata = metadata

In [3]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [18]:
from llama_index.core import Settings
from transformers import AutoTokenizer, AutoModelForCausalLM

Settings.llm=model
Settings.embed_model=embed_model
Settings.tokenzier=tokenizer

In [19]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
#chroma_collection = chroma_client.create_collection("quickstart")

# load documents
documents = SimpleDirectoryReader("txt_db").load_data()

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [20]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)
# Query Data
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("경북대 공지사항 사이트 알려줘")
print(response)

# BM25 Chroma 

In [ ]:
%pip install llama-index-retrievers-bm25
!pip install KoNLPy

In [1]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, FilterOperator
from llama_index.core import VectorStoreIndex
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.response.notebook_utils import display_source_node
import nest_asyncio

/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [31]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|>#         
        당신은 경북대학교 정보 안내 챗봇으로, 질문에 대한 답을 해주어야합니다.
        Casual Answer로 답변 형식을 지정합니다.
        당신은 학습된 내용 안에서 질문에 답변해야합니다.
        학습된 내용이 없을 경우에는, 연결된 DB내용을 참조하도록 합니다.
        학습된 내용은 datasets항목 안에 있는 것을 뜻하며,
        DB 연결은 아직 하지 않아, DB를 참조할 경우 "내용을 찾지 못했습니다."라고 답하여야합니다.
        당신은 스스로 답변 가치를 생성할 수 있습니다.
        null이나 빈 값은 반환하지 않습니다.
        url 반환 시, <SYSTEM>을 붙이지 않습니다.

        Question: 크누큐브 사이트 알려줘
        Answer: https://knucube.knu.ac.kr/

        Question: 국어국문학과 전공 커리큘럼 사이트를 알려줘
        Answer: https://home.knu.ac.kr/HOME/knujob/sub.htm?nav_code=knu1668399325&code=dept1&dept=dept-korean
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [10]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

model= HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun",
    model_name="Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
)

/raid/coss35/anaconda3/envs/llm/lib/python3.9/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/raid/coss35/anaconda3/envs/llm/lib/python3.9/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/raid/coss35/anaconda3/envs/llm/lib/python3.9/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/raid/coss35/anaconda3/envs/llm/lib/python3.9/site-packages/pydantic/_internal/_

In [11]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [12]:
from llama_index.core import Settings

Settings.llm=model
Settings.embed_model=embed_model
Settings.chunk_size = 512

In [15]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader("txt_db").load_data()

In [16]:
from llama_index.core.node_parser import SentenceSplitter

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)

In [17]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [18]:
from llama_index.retrievers.bm25 import BM25Retriever
from konlpy.tag import Okt  # konlpy의 Okt 형태소 분석기 import

# 한국어 불용어 리스트
korean_stopwords = ['이', '그', '저', '있다', '하다', '것', '들', '때', '등', '에서']

# konlpy 기반의 Okt 토크나이저를 활용하여 텍스트를 한국어로 처리하는 BM25 리트리버 설정
class KonlpyTokenizer:
    def __init__(self):
        self.tokenizer = Okt()

    def tokenize(self, text):
        # konlpy의 morphs 함수를 사용하여 형태소 단위로 토큰화
        tokens = self.tokenizer.morphs(text)
        # 불용어 제거
        tokens = [token for token in tokens if token not in korean_stopwords]
        return tokens

# Korean BM25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2,
    stemmer=None,  # PyStemmer 대신 직접 토크나이저에서 처리
    language=None,  # 언어 기본 처리를 비활성화
    tokenizer=KonlpyTokenizer(),  # konlpy 기반의 토크나이저
)

The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.


In [19]:
bm25_retriever.persist("./bm25_retriever")

loaded_bm25_retriever = BM25Retriever.from_persist_dir("./bm25_retriever")

Finding newlines for mmindex: 100%|███████████████████████████████████████████████████| 410k/410k [00:00<00:00, 177MB/s]


In [20]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("dense_vectors")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(
    docstore=docstore, vector_store=vector_store
)

index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)

In [21]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=2
        ),
    ],
    num_queries=1,
    use_async=True,
)

In [19]:
retrieved_nodes = bm25_retriever.retrieve("경북대학교 대학시설공간 종합관리시스템")
for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 99b1b33c-97cf-402a-a7f7-6908e25fbd3c<br>**Similarity:** 5.2211761474609375<br>**Text:** 스마트러닝센터의 원격 콘텐츠 수업 개발과정은 다음과 같습니다 : 원격수업 콘텐츠 개발 신청 접수 및 선정평가 → 오리엔테이션 실시 → 교안 저작권 검토 → 영상 촬영 및 편집 → 최종 영상 출력물 검토 및 안내 → LCMS 탑재 안내 및 지원

스마트러닝센터에서는 교육용 콘텐츠 영상 제작 또한 지원하고 있습니다.

스마트러닝센터의 제작 지원을 신청할 수 있는 대상은 경북대학교 소속 전임교원 및 기관입니다.

스마트러닝센터의 스튜디오는 3D, 호리즌, 블랙 스튜디오 등의 최첨단 콘텐츠 제작실입니다.

=============================

https://ufis.knu.ac.kr/ufis/index.do

**경북대학교 대학시설공간 종합관리시스템**

대학시설공간 종합관리시스템 사이트는 공간현황관리 및 공간조정관리, 공간비용관리, 시설정보관리, 사업수행관리, 전자도면 관리, 시스템관리의 7가지 갈래로 나뉩니다.<br>

**Node ID:** a9612121-ae28-458d-ba83-b0dbf50f8325<br>**Similarity:** 5.2211761474609375<br>**Text:** =============================

https://ufis.knu.ac.kr/ufis/index.do

**경북대학교 대학시설공간 종합관리시스템**

대학시설공간 종합관리시스템 사이트는 공간현황관리 및 공간조정관리, 공간비용관리, 시설정보관리, 사업수행관리, 전자도면 관리, 시스템관리의 7가지 갈래로 나뉩니다.

=============================

https://safe.knu.ac.kr/

**경북대학교 연구실안전관리센터**

연구실안전관리센터는 세계 초일류 연구중심대학이 되기 위해 연구활동을 수행하는 교수, 연구원, 대학(원)생 등의 연구활동종사자가 안전하고 쾌적한 환경에서 연구할 수 있도록 하기 위해 설립되었습니다.

연구실안전관리센터는 연구실에서 발생하는 각종 안전사고에 대하여 대비·대응·예방 등 적극 지원을 통해 무재해·무사고를 도모하고자합니다.

연구실안전관리센터에서는 연구실 안전에 관한 기본 걔획을 수립하고, 안전과 관련된 정기점검, 특별안전점검 및 정밀안전진단 관련 업무를 수행합니다.<br>

In [22]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever)

In [30]:
response = query_engine.query("비교과 교육과정 통합관리 시스템  알려줘")
#print(type(response))
print(response)

https://knucube.knu.ac.kr/HOME/global/sub.htm?nav_code=glo1549935238</ASSISTANT|>


In [6]:
from llama_index.core import Settings
from llama_index.core import PromptTemplate
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
from konlpy.tag import Okt  # konlpy의 Okt 형태소 분석기 import
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import QueryFusionRetriever
import chromadb
import nest_asyncio
from llama_index.core.query_engine import RetrieverQueryEngine


# Define a directory to persist index data
PERSIST_DIR = "./storage"

# 한국어 불용어 리스트
korean_stopwords = ['이', '그', '저', '있다', '하다', '것', '들', '때', '등', '에서']

        # konlpy 기반의 Okt 토크나이저를 활용하여 텍스트를 한국어로 처리하는 BM25 리트리버 설정
class KonlpyTokenizer:
    def __init__(self):
        self.tokenizer = Okt()
        
    def tokenize(self, text):
        # konlpy의 morphs 함수를 사용하여 형태소 단위로 토큰화
        tokens = self.tokenizer.morphs(text)
        # 불용어 제거
        tokens = [token for token in tokens if token not in korean_stopwords]
        return tokens


class RAGService:
    def __init__(self):
        ###MODEL 구축##################
        system_prompt="""
        <|SYSTEM|>#         
        당신은 경북대학교 정보 안내 챗봇으로, 질문에 대한 답을 해주어야합니다.
        Casual Answer로 답변 형식을 지정합니다.
        당신은 학습된 내용 안에서 질문에 답변해야합니다.
        학습된 내용이 없을 경우에는, 연결된 DB내용을 참조하도록 합니다.
        학습된 내용은 datasets항목 안에 있는 것을 뜻하며,
        DB 연결은 아직 하지 않아, DB를 참조할 경우 "내용을 찾지 못했습니다."라고 답하여야합니다.
        당신은 스스로 답변 가치를 생성할 수 있습니다.
        null이나 빈 값은 반환하지 않습니다.
        url 반환 시, <SYSTEM>을 붙이지 않습니다.

        Question: 크누큐브 사이트 알려줘
        Answer: https://knucube.knu.ac.kr/

        Question: 국어국문학과 전공 커리큘럼 사이트를 알려줘
        Answer: https://home.knu.ac.kr/HOME/knujob/sub.htm?nav_code=knu1668399325&code=dept1&dept=dept-korean
        """
        # This will wrap the default prompts that are internal to llama-index
        query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

        model= HuggingFaceLLM(
            context_window=2048,
            max_new_tokens=256,
            generate_kwargs={"temperature": 0.25, "do_sample": False},
            system_prompt=system_prompt,
            query_wrapper_prompt=query_wrapper_prompt,
            tokenizer_name="Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun",
            model_name="Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun",
            device_map="auto",
            tokenizer_kwargs={"max_length": 2048},
            # uncomment this if using CUDA to reduce memory usage
            # model_kwargs={"torch_dtype": torch.float16}
        )
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
        
        Settings.llm=model
        Settings.embed_model=embed_model
        Settings.chunk_size = 512
        ########RAG##########################
        # load documents
        documents = SimpleDirectoryReader("txt_db").load_data()
        
        # initialize node parser
        splitter = SentenceSplitter(chunk_size=512)
        
        nodes = splitter.get_nodes_from_documents(documents)

        konlpytokenizer=KonlpyTokenizer()
        
        # Korean BM25 retriever
        bm25_retriever = BM25Retriever.from_defaults(
            nodes=nodes,
            similarity_top_k=2,
            stemmer=None,  # PyStemmer 대신 직접 토크나이저에서 처리
            language=None,  # 언어 기본 처리를 비활성화
            tokenizer=KonlpyTokenizer(),  # konlpy 기반의 토크나이저
        )
        
        bm25_retriever.persist("./bm25_retriever")
        loaded_bm25_retriever = BM25Retriever.from_persist_dir("./bm25_retriever")

        docstore = SimpleDocumentStore()
        docstore.add_documents(nodes)
        
        db = chromadb.PersistentClient(path="./chroma_db")
        chroma_collection = db.get_or_create_collection("dense_vectors")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        
        storage_context = StorageContext.from_defaults(
            docstore=docstore, vector_store=vector_store
        )
        
        #index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)
        self.index=VectorStoreIndex(nodes=nodes, storage_context=storage_context)
        #nest_asyncio.apply()

        self.retriever = QueryFusionRetriever(
            [
                self.index.as_retriever(similarity_top_k=2),
                BM25Retriever.from_defaults(
                    docstore=self.index.docstore, similarity_top_k=2
                ),
            ],
            num_queries=1,
            use_async=True,
        )
    def query(self, query: str) -> str:
        query_engine = RetrieverQueryEngine(self.retriever)  
        response = query_engine.query(query)
        return str(response)

        

rag_service=RAGService()
       
query_result=rag_service.query("경북대 컴학 알려줘 ")
print(query_result)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:22<00:00,  5.53s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/raid/coss35/anaconda3/envs/llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
DEBUG:bm25s:Building index from IDs objects
Finding newlines for mmindex: 100%|███████████████████████████████████████████████████| 4

https://cse.knu.ac.kr/sub03/sub03_01.html</ASSISTANT|>


- DB 구성 깔끔하게 해야할듯.. ??


## Retriever Query Engine with Custom Retrievers - Simple Hybrid Search